## 找出看漲/看跌關鍵字

In [1]:
import numpy as np
import pandas as pd
import jieba
import jieba.analyse

jieba.set_dictionary('data/dict.idkrsi.txt')    #預設字典改繁體
jieba.analyse.set_stop_words('data/stopword.goatwang.kang.txt') #載入暫停字

df_down = pd.read_csv('data/pre-3_day_articles_down.csv',low_memory=True)
df_up = pd.read_csv('data/pre-3_day_articles_up.csv',low_memory=True)
df_down.drop(['Unnamed: 0'],axis=1, inplace=True) 
df_up.drop(['Unnamed: 0'],axis=1, inplace=True) 
df_up.head()

,post_time,title,content,type
0,2016-05-22 08:45:00,新低量隱憂 後520時代 台股關注新政府概念股！,台股本周在新政府上台及總統就職大典收周線，儘管市場一度擔心兩岸關係變數對盤面的影響，但在就職...,漲
1,2016-05-22 09:00:00,《各報要聞》國台辦、海協會強勢回應，堅持九二共識兩岸才能互動,在評價總統蔡英文就職演說為「沒有完成的答卷」後，中國大陸國台辦及海協會昨（21）日上午雙雙發...,漲
2,2016-05-22 09:01:00,《各報要聞》新教長guts撂話，私校退場董事淨身出戶,少子化衝擊高教，大學退場迫在眉睫，教育部長潘文忠一上任就宣告，將設置50億元基金，作為私校退...,漲
3,2016-05-22 09:01:00,《各報要聞》520後兩岸交鋒，台股恐失守8千,蔡英文總統520演說後，國台辦連續二日發表強硬聲明，尤其昨日沒有「九二共識」將中斷兩岸協商機...,漲
4,2016-05-22 09:05:00,《政治》蔡第一刀，駐美代表奉令6月5日離任,蔡英文政府上任第一天，突傳駐美代表沈呂巡奉指示6月5日離任。時間之緊迫，既不符慣例，也不符法...,漲


### 把停用詞存成list比對

In [16]:
stopwords = []
with open('data/stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))
        
stopwords[:20]

['\ufeff\ufeff,',
 '',
 '、',
 '。',
 '“',
 '”',
 '《',
 '》',
 '！',
 '，',
 '：',
 '；',
 '？',
 '人民',
 '末##末',
 '啊',
 '阿',
 '哎',
 '哎呀',
 '哎喲']

### 結巴切詞且過濾停用字

In [41]:
title = []
# rec[0]是index rec[1]是row (類似axis)
# rec[1]['title']  dataframe由上到下一一挑出來
for rec in df_up.iterrows():
    templist = str(rec[1]['title']).replace('\n','').replace(',',' ')
    templist = jieba.cut(templist, cut_all=False)
    seglist=list(templist)
    newseglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist))
    title.append(' '.join(newseglist))

df_up['title_cut']=title


content = []
# rec[0]是index rec[1]是row (類似axis)
# rec[1]['content']  dataframe由上到下一一挑出來
for rec in df_up.iterrows():
    templist = str(rec[1]['content']).replace('\n','').replace(',',' ')
    templist = jieba.cut(templist, cut_all=False)
    seglist=list(templist)
    newseglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist))
    content.append(' '.join(newseglist))
    

df_up['content_cut']=content
df_up.head()

,post_time,title,content,type,title_cut,content_cut
0,2016-05-22 08:45:00,新低量隱憂 後520時代 台股關注新政府概念股！,台股本周在新政府上台及總統就職大典收周線，儘管市場一度擔心兩岸關係變數對盤面的影響，但在就職...,漲,新 低量 隱憂 520 時代 台股 關注 新 政府 概念股,台股 本周 新 政府 上台 總統 就職 大典 收周線 市場 一度 擔心 兩岸 關係 變數 盤...
1,2016-05-22 09:00:00,《各報要聞》國台辦、海協會強勢回應，堅持九二共識兩岸才能互動,在評價總統蔡英文就職演說為「沒有完成的答卷」後，中國大陸國台辦及海協會昨（21）日上午雙雙發...,漲,各報 要聞 國台辦 海協會 強勢 回應 堅持 九二 共識 兩岸 互動,評價 總統 蔡英文 就職 演說 完成 答卷 中國 大陸 國台辦 海協會 昨 21 上午 雙雙...
2,2016-05-22 09:01:00,《各報要聞》新教長guts撂話，私校退場董事淨身出戶,少子化衝擊高教，大學退場迫在眉睫，教育部長潘文忠一上任就宣告，將設置50億元基金，作為私校退...,漲,各報 要聞 新 教長 guts 撂話 私校 退場 董事 淨身 戶,少子化 衝擊 高教 大學 退場 迫在眉睫 教育部長 潘文忠 上任 宣告 設置 50 億元 基...
3,2016-05-22 09:01:00,《各報要聞》520後兩岸交鋒，台股恐失守8千,蔡英文總統520演說後，國台辦連續二日發表強硬聲明，尤其昨日沒有「九二共識」將中斷兩岸協商機...,漲,各報 要聞 520 兩岸 交鋒 台股 恐 失守,蔡英文 總統 520 演說 國台辦 連續 二日 發表 強硬 聲明 尤其 昨日 九二 共識 中...
4,2016-05-22 09:05:00,《政治》蔡第一刀，駐美代表奉令6月5日離任,蔡英文政府上任第一天，突傳駐美代表沈呂巡奉指示6月5日離任。時間之緊迫，既不符慣例，也不符法...,漲,政治 蔡第一刀 駐美 代表 奉令 離任,蔡英文 政府 上任 第一天 突傳 駐美 代表 沈呂巡 奉 指示 離任 時間 緊迫 不符 慣例...


In [290]:
type(df_up['title_cut'][0])

str

### 針對標題做 tfidf , 挑出分數前 1000 字詞

In [175]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

title_corpus = list(df_up['title_cut'])
title_corpus

vectorizer = CountVectorizer()
frequency = vectorizer.fit_transform(title_corpus)

transformer = TfidfTransformer()
tfidf = transformer.fit_transform(frequency)
word = vectorizer.get_feature_names() # 詞的矩陣

weight = tfidf.toarray()#將tf-idf矩陣抽取出来，元素a[i][j]表示j詞在i個文件中的tf-idf權重

title_df = pd.DataFrame(weight, columns=word)
title_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1667 entries, 0 to 1666
Columns: 4736 entries, 00 to 龍頭
dtypes: float64(4736)
memory usage: 60.2 MB


In [177]:
title_df.head(2)

,00,000,0050,005930,00672l,00692,00693u,01,02,03,...,點關,黨產,黨職,黨證,黯淡,鼓勵,齊攻,龍井,龍捲風,龍頭
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
dftest = title_df.loc[0,:] ##測試

In [137]:
##測試
mapp = dict(zip((dftest[dftest.values>0].sort_values(ascending=False)).index,
(dftest[dftest.values>0].sort_values(ascending=False)).values))

mapp

{'隱憂': 0.39167461837587264,
 '時代': 0.39167461837587264,
 '低量': 0.39167461837587264,
 '520': 0.3762830964191371,
 '關注': 0.36434450448573713,
 '概念股': 0.36434450448573713,
 '政府': 0.32725985072919855,
 '台股': 0.159972389843866}

In [283]:
title_dic = {}
for index,row in title_df.iterrows():
    rows = title_df.loc[index,:] #每一列
    key = (row[row.values>0].sort_values(ascending=False)).index # 把權重大於0的詞挑出
    value = (row[row.values>0].sort_values(ascending=False)).values
    mapping = dict(zip(key,value)) 
    title_dic.update(mapping) #更新
    
title_dic

{'隱憂': 0.2947734433374898,
 '時代': 0.38543840851621636,
 '低量': 0.4976526145350935,
 '520': 0.46025097879753574,
 '關注': 0.44017386135438763,
 '概念股': 0.44495918242834537,
 '政府': 0.45183939475445856,
 '台股': 0.18997505185326863,
 '海協會': 0.34747402677347605,
 '國台辦': 0.34747402677347605,
 '堅持': 0.3398167912450631,
 '回應': 0.38221987010426434,
 '九二': 0.4463476348772724,
 '共識': 0.42880764351649003,
 '互動': 0.35993807803477684,
 '強勢': 0.4078388363211262,
 '兩岸': 0.3658842720532007,
 '各報': 0.21893962011477697,
 '要聞': 0.20072634964015598,
 '退場': 0.3650407315358645,
 '私校': 0.3650407315358645,
 '淨身': 0.3650407315358645,
 '教長': 0.3650407315358645,
 '撂話': 0.3650407315358645,
 'guts': 0.3650407315358645,
 '董事': 0.3763177271512989,
 '交鋒': 0.5071268804511541,
 '失守': 0.4927368106904878,
 '駐美': 0.44384255572975795,
 '離任': 0.44384255572975795,
 '蔡第一刀': 0.44384255572975795,
 '奉令': 0.44384255572975795,
 '代表': 0.29880635607004,
 '政治': 0.2013324944797449,
 '雙巴': 0.465588989846057,
 '過境': 0.465588989846057,
 '友邦': 

In [284]:
len(title_dic)

4736

In [278]:
## 排序後，挑出前 1000個字詞

In [285]:
sorted_title = sorted(title_dic.items(), key=lambda kv: kv[1])
sorted_title.reverse() #反轉

sorted_title_1000 = sorted_title[:1000]
sorted_title_1000

[('防癌險', 1.0),
 ('永豐餘', 1.0),
 ('台郡', 1.0),
 ('邦特', 1.0),
 ('書籍', 1.0),
 ('退休', 1.0),
 ('鈊象', 1.0),
 ('6123', 0.8966860389785241),
 ('山林', 0.8966860389785241),
 ('統一超', 0.8868490463917621),
 ('亞泥', 0.8813060903477435),
 ('不想', 0.8490059021389077),
 ('冷清清', 0.8352121398357325),
 ('新普', 0.8275776332102315),
 ('還沒漲', 0.8132218552209219),
 ('封關日', 0.806074345633226),
 ('soxx', 0.8007133656190472),
 ('岸風電', 0.7966196799270964),
 ('撞牆', 0.7966196799270964),
 ('比賽', 0.7930594477229537),
 ('漲升', 0.7887486290761102),
 ('閒聊', 0.7850560689748894),
 ('環球晶', 0.7801516930507691),
 ('辦幾張', 0.773050531449205),
 ('質押', 0.7727080376636456),
 ('房貸', 0.769371549508316),
 ('財報', 0.7687752281168966),
 ('分散', 0.7608915602332628),
 ('鋁價', 0.759007272256197),
 ('價漲', 0.759007272256197),
 ('策略', 0.7549585733119455),
 ('造紙', 0.75300890369803),
 ('加權', 0.7511137783227229),
 ('致富', 0.7501904575464966),
 ('通知書', 0.7413307579961738),
 ('基富通', 0.739499833343325),
 ('郵局', 0.7341316173032936),
 ('承銷', 0.732908835168997

### 針對內文做 tfidf , 挑出分數大於 平均權重 的字詞

In [155]:
content_corpus = list(df_up['content_cut'])

vectorizer = CountVectorizer()
frequency = vectorizer.fit_transform(content_corpus)

transformer = TfidfTransformer()
tfidf_content = transformer.fit_transform(frequency)
word_content = vectorizer.get_feature_names() # 詞的矩陣

weight_content = tfidf_content.toarray()#將tf-idf矩陣抽取出来，元素a[i][j]表示j詞在i個文件中的tf-idf權重

content_df = pd.DataFrame(weight_content, columns=word_content)

In [286]:
content_dic = {}
for index,row in content_df.iterrows():
    rows = content_df.loc[index,:] #每一列
    key = (row[row.values>0].sort_values(ascending=False)).index # 把權重大於0.5的詞挑出
    value = (row[row.values>0].sort_values(ascending=False)).values
    mapping = dict(zip(key,value)) 
    content_dic.update(mapping) #更新
    
content_dic

{'預期': 0.03398798494582204,
 '持續': 0.021869481411858325,
 '汽車': 0.024490285897116198,
 '表現': 0.2946776458136317,
 '郭修': 0.14728533084531864,
 '富蘭克林華美': 0.15847458695750352,
 '下半年': 0.10179723514657595,
 '近期': 0.050993596944600275,
 '零組件': 0.058174597895698066,
 '相對': 0.03117331682679834,
 '供應鏈': 0.03505497814760796,
 '獲利': 0.03485284670357587,
 '成長': 0.02461441860552761,
 '表示': 0.018322289272296784,
 '台股': 0.13621986120250357,
 '經理人': 0.06955756867398216,
 '電子': 0.13779897623649276,
 '生技': 0.12402588996326865,
 '因素': 0.025348706878464047,
 '施生元': 0.09819022056354576,
 '市場': 0.018454162236006377,
 '財報': 0.14474869987333064,
 '謹慎': 0.12406597259181154,
 '基金': 0.18570398315272535,
 '食品股': 0.16540821458517385,
 '將是': 0.061306284065769004,
 '影響': 0.03277984736988842,
 '醫療': 0.1035869502025768,
 '季及': 0.049071399329447714,
 '周書玄': 0.15847458695750352,
 '醫材': 0.3243038029400777,
 '看法': 0.4429113688089803,
 'fed': 0.047900330643764685,
 '政策': 0.07120887871387173,
 '留意': 0.13844371786837692,
 '

In [287]:
len(content_dic)

33427

In [291]:
sorted_content = sorted(content_dic.items(), key=lambda kv: kv[1])
sorted_content.reverse() #反轉

sorted_content_1000 = sorted_content[:1000]
sorted_content_1000

[('nan', 1.0),
 ('1515151516', 0.8717761846132527),
 ('勤美', 0.8179625141888306),
 ('千股', 0.8004297728164099),
 ('聯鈞', 0.7911772482145939),
 ('dragoncfe168', 0.7555616608930527),
 ('減資', 0.7421580667576297),
 ('ssd', 0.7271386252533435),
 ('買回', 0.7119521901507261),
 ('李晨', 0.7084286590515745),
 ('fii', 0.7079809673345521),
 ('gap', 0.6996592695504715),
 ('信用戶', 0.6953979007404502),
 ('新普', 0.6924183325688475),
 ('提款', 0.6869098120259708),
 ('中產', 0.6855526016511045),
 ('變天', 0.6843007729556623),
 ('喬福', 0.6783225438454504),
 ('實支', 0.6740625055131948),
 ('老爹', 0.6739821687970884),
 ('博通', 0.6678716376596775),
 ('gpif', 0.6666142249766885),
 ('net', 0.6591236565019462),
 ('黃豆', 0.6495189802746382),
 ('受益人', 0.6460121609015501),
 ('5534', 0.6449389611810844),
 ('南仁湖', 0.6392244734148123),
 ('哈林', 0.6386929946929889),
 ('張兆順', 0.6360666712461175),
 ('美光', 0.6309161974756076),
 ('禮拜', 0.6297012863478055),
 ('警示股', 0.6273471346356355),
 ('張志鵬', 0.6231123092034012),
 ('put', 0.62282433989871